In [9]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy.stats import nbinom, gamma, chi2_contingency
from scipy.special import digamma
from scipy.optimize import minimize


In [10]:
# define root directory where FAERS datasets are located
root_dir = '/Users/jodie/Documents/BMI 212/faers-cohort-generation/FAERS-data-toolkit-master/FAERSdata'
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

# adjust range from 2013 to 2025
all_quarters = [f"{str(y)[2:]}{q}" for y in range(2013, 2026) for q in quarters]

# limit to only those quarters that exist (sanity check)
existing_quarters = [q for q in all_quarters if os.path.exists(os.path.join(root_dir, f"DEMO{q}.txt"))]

# store DataFrames in a dict
data = {}

for q in existing_quarters:
    try:
        data[q] = {
            'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'reac': pd.read_csv(os.path.join(root_dir, f"REAC{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'outc': pd.read_csv(os.path.join(root_dir, f"OUTC{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'indi': pd.read_csv(os.path.join(root_dir, f"INDI{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'rpsr': pd.read_csv(os.path.join(root_dir, f"RPSR{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            'ther': pd.read_csv(os.path.join(root_dir, f"THER{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
        }
        print(f"Loaded {q} successfully.")
    except Exception as e:
        print(f"Error loading data for {q}: {e}")


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:17: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:18: DtypeWarning: Columns (9,10,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 13Q1 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:18: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 13Q2 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:17: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:18: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


...

Loaded 24Q3 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:18: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 24Q4 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_40078/2161774026.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 25Q1 successfully.


In [12]:
## start_year = input("Enter the starting year to query (e.g. 2017): ") 
## start_quarter = input("Enter the starting quarter to query (e.g. Q1): ") 
## end_year = input("Enter the starting year to query (e.g. 2021): ") 
## end_quarter = input("Enter the starting year to query (e.g. Q4): ") 

start_year = 2017
start_quarter = 'Q1'
end_year = 2021 
end_quarter = 'Q4'


In [13]:
def generate_periods(start_year, start_quarter, end_year, end_quarter):
    periods = []
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    
    start_year = int(start_year)
    end_year = int(end_year)
    
    # Loop through the years and quarters to generate all periods in the range
    for year in range(start_year, end_year + 1):
        start_qtr = start_quarter if year == start_year else 'Q1'
        end_qtr = end_quarter if year == end_year else 'Q4'
        
        for qtr in quarters[quarters.index(start_qtr):quarters.index(end_qtr) + 1]:
            periods.append(f"{str(year)[-2:]}{qtr}")
    
    return periods

In [14]:
# Generate periods based on user input
periods = generate_periods(start_year, start_quarter, end_year, end_quarter)

# List of table types to be processed (can easily be extended)
table_types = ['demo', 'drug', 'reac', 'outc', 'indi', 'rpsr', 'ther']

# Initialize a dictionary to store lists of DataFrames for each table
data_dict = {table: [] for table in table_types}

# Loop through each period and collect data for each table type
for period in periods:
    if period in data:  # Check if data for the period exists
        for table in table_types:
            if table in data[period]:
                data_dict[table].append(data[period][table])
    else:
        print(f"Warning: No data available for {period}")

# Concatenate data for each table type into a single DataFrame
merged_data = {
    table: pd.concat(data_dict[table], ignore_index=True) if data_dict[table] else pd.DataFrame()
    for table in table_types
}

In [16]:
demo = merged_data['demo']
drug = merged_data['drug']
reac = merged_data['reac']
outc = merged_data['outc']
indi = merged_data['indi']
rpsr = merged_data['rpsr']
ther = merged_data['ther']

In [17]:
demo

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,100039237,10003923,7,F,2012.0,20170111.0,20140312,20170119,EXP,NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
1,1000588614,10005886,14,F,20140116.0,20170126.0,20140313,20170201,EXP,NaN,...,NaN,M,Y,90.00,KG,20170201.0,NaN,MD,IL,IL
2,100063033,10006303,3,F,NaN,20140304.0,20140313,20170227,EXP,NaN,...,A,F,Y,NaN,NaN,20170227.0,NaN,OT,CA,CA
3,100065923,10006592,3,F,20170211.0,20170214.0,20140313,20170217,EXP,NaN,...,E,F,Y,NaN,NaN,20170217.0,NaN,OT,US,US
4,100085064,10008506,4,F,20140402.0,20170214.0,20140313,20170220,EXP,NaN,...,A,F,Y,NaN,NaN,20170220.0,NaN,OT,US,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8399447,99343632,9934363,2,F,NaN,20140218.0,20140228,20211201,EXP,CA-NOVOPROD-400774,...,NaN,F,Y,95.00,KG,20211201.0,NaN,CN,CA,CA
8399448,99371397,9937139,7,F,20190101.0,20211002.0,20140301,20211007,EXP,NaN,...,NaN,F,Y,81.60,KG,20211007.0,NaN,CN,CA,CA
8399449,994329779,9943297,79,F,20130506.0,20211104.0,20140303,20211113,EXP,NaN,...,NaN,M,Y,NaN,NaN,20211113.0,NaN,HP,CA,CA
8399450,999647512,9996475,12,F,20140101.0,20211215.0,20140311,20211225,EXP,NaN,...,A,F,Y,NaN,NaN,20211225.0,NaN,MD,US,US


In [18]:
demo['occp_cod']

0          OT
1          MD
2          OT
3          OT
4          OT
           ..
8399447    CN
8399448    CN
8399449    HP
8399450    MD
8399451    MD
Name: occp_cod, Length: 8399452, dtype: object

In [3]:
# Processes drug df
drug = drug[pd.notnull(drug['drugname'])]  # Drops Nulls
drug['drugname'] = drug['drugname'].str.strip().str.lower()  # Stips whitespace, Transforms to lowercase
drug = drug[~drug['drugname'].isin(['unknown'])]  # Drops unknowns
drug['drugname'] = drug['drugname'].str.replace('\\', '/')  # Standardizes slashes to '/'
drug['drugname'] = drug['drugname'].map(
    lambda x: x[:-1] if str(x).endswith(".") else x)  # Removes periods at the end of drug names


In [4]:
# Process reac df 
reac = reac[pd.notnull(reac['pt'])] # Drops Nulls
reac['pt'] = reac['pt'].str.strip().str.lower()  # Transforms to lowercase
reac = reac[~reac['pt'].isin(['unknown'])]  # Drops unknowns
reac['pt'] = reac['pt'].map(
    lambda x: x[:-1] if str(x).endswith(".") else x)  # Removes periods at the end of drug names


In [5]:
# Process demo df 
demo = demo[pd.notnull(demo['age'])]
demo = demo[demo.age_cod != 'dec'].reset_index(drop=True) 
demo['age'] = demo['age'].apply(pd.to_numeric, errors='coerce')
demo['age'] = np.where(demo['age_cod'] == 'MON', demo['age'] * 1 / 12, demo['age'])  # mounth
demo['age'] = np.where(demo['age_cod'] == 'WK', demo['age'] * 1 / 52, demo['age'])  # week
demo['age'] = np.where(demo['age_cod'] == 'DY', demo['age'] * 1 / 365, demo['age'])  # day
demo['age'] = np.where(demo['age_cod'] == 'HR', demo['age'] * 1 / 8760, demo['age'])  # hour
demo = demo.drop(['age_cod'], axis=1)


In [19]:
drug.head(5)

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100039237,10003923,1,PS,JAKAFI,RUXOLITINIB,1,Oral,"20 MG, BID",NaN,NaN,U,U,A81294Q,NaN,202192.0,20.0,MG,TABLET,BID
1,100039237,10003923,2,SS,JAKAFI,RUXOLITINIB,1,Oral,"7.5 MG, BID",NaN,NaN,U,U,A79945S,NaN,202192.0,7.5,MG,TABLET,BID
2,100039237,10003923,3,SS,JAKAFI,RUXOLITINIB,1,Oral,15 MG,NaN,NaN,U,U,NaN,NaN,202192.0,NaN,NaN,TABLET,NaN
3,100039237,10003923,4,C,ASPIRIN.,ASPIRIN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100039237,10003923,5,C,PLAVIX,CLOPIDOGREL BISULFATE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TABLET,NaN


In [20]:
demo_latest = demo.sort_values('caseversion', ascending=False).drop_duplicates(subset='caseid', keep='last')

drug = pd.merge(drug, demo_latest, on='primaryid', how='left')


In [21]:
drug

,primaryid,caseid_x,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,100039237,10003923,1,PS,JAKAFI,RUXOLITINIB,1,Oral,"20 MG, BID",NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
1,100039237,10003923,2,SS,JAKAFI,RUXOLITINIB,1,Oral,"7.5 MG, BID",NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
2,100039237,10003923,3,SS,JAKAFI,RUXOLITINIB,1,Oral,15 MG,NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
3,100039237,10003923,4,C,ASPIRIN.,ASPIRIN,1,NaN,NaN,NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
4,100039237,10003923,5,C,PLAVIX,CLOPIDOGREL BISULFATE,1,NaN,NaN,NaN,...,NaN,F,Y,62.13,KG,20170119.0,NaN,OT,US,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35060766,999647512,9996475,9,C,BIOTIN,BIOTIN,1,NaN,NaN,NaN,...,A,F,Y,NaN,NaN,20211225.0,NaN,MD,US,US
35060767,999647512,9996475,10,C,ADDERALL,AMPHETAMINE ASPARTATE\AMPHETAMINE SULFATE\DEXT...,1,NaN,NaN,NaN,...,A,F,Y,NaN,NaN,20211225.0,NaN,MD,US,US
35060768,99978616,9997861,1,PS,TOVIAZ,FESOTERODINE FUMARATE,1,Oral,"4 mg, daily",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35060769,99978616,9997861,2,C,ATENOLOL,ATENOLOL,1,NaN,UNK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
query_drug = 'niraparib' 

# Finds reports related to the query drug 
query_drug_df = drug[
    drug['drugname'].str.lower().str.contains(query_drug, na=False) |
    drug['prod_ai'].str.lower().str.contains(query_drug, na=False)
]

# Subsets to primary suspect drugs
query_drug_ps = query_drug_df[query_drug_df['role_cod'] == 'PS']



In [23]:
query_drug_ps

,primaryid,caseid_x,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
1835316,134974491,13497449,1,PS,NIRAPARIB,NIRAPARIB,1,Oral,"300 MG, QD",15300.0,...,NaN,F,Y,61.8,KG,20170428.0,NaN,MD,CA,CA
1835492,134975003,13497500,1,PS,NIRAPARIB,NIRAPARIB,1,Oral,"300 MG, QD",NaN,...,NaN,F,Y,77.5,KG,20170602.0,NaN,MD,ES,ES
1835504,134975091,13497509,1,PS,NIRAPARIB,NIRAPARIB,1,Oral,"300 MG, QD",4500.0,...,NaN,F,Y,58.6,KG,20170428.0,NaN,MD,US,US
1835532,134975182,13497518,1,PS,NIRAPARIB,NIRAPARIB,1,Oral,"300 MG, QD",17200.0,...,NaN,F,Y,NaN,NaN,20170531.0,NaN,MD,US,US
1835549,134975243,13497524,1,PS,NIRAPARIB,NIRAPARIB,1,Oral,"100 MG, QD",43800.0,...,NaN,F,Y,72.6,KG,20170510.0,NaN,MD,US,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35042207,202637551,20263755,1,PS,ZEJULA,NIRAPARIB,1,NaN,"100 mg, qd",NaN,...,NaN,F,Y,NaN,NaN,20211231.0,NaN,CN,US,US
35043400,202640861,20264086,1,PS,NIRAPARIB,NIRAPARIB,1,NaN,"200 mg, qd",NaN,...,NaN,F,Y,63.5,KG,20211231.0,NaN,CN,US,US
35043444,202640971,20264097,1,PS,ZEJULA,NIRAPARIB,1,Oral,200 milligram,NaN,...,NaN,F,Y,NaN,NaN,20211231.0,NaN,MD,JP,JP
35045716,202647661,20264766,1,PS,NIRAPARIB,NIRAPARIB,1,NaN,"100mg, qd",NaN,...,NaN,F,Y,NaN,NaN,20211231.0,NaN,CN,US,US


In [24]:

# Get AE counts for cases where query drug is mentioned
query_drug_ids = query_drug_df['primaryid'].unique()
query_drug_reac = reac[reac['primaryid'].isin(query_drug_ids)]
ae_counts = query_drug_reac['pt'].value_counts().reset_index()
ae_counts.columns = ['Adverse_Event', 'Count']

# Get AE counts for cases where query drug is the primary suspect 
ps_ids = query_drug_ps['primaryid'].unique()
ps_reac = reac[reac['primaryid'].isin(ps_ids)]
ps_ae_counts = ps_reac['pt'].value_counts().reset_index()
ps_ae_counts.columns = ['Adverse_Event', 'Count']

# Get AE counts for cases where query drug is not mentioned
non_query_drug = drug[~drug['primaryid'].isin(query_drug_ids)]
non_ps_drug = drug[~drug['primaryid'].isin(ps_ids)]
non_query_ids = non_query_drug['primaryid'].unique()
non_query_reac = reac[reac['primaryid'].isin(non_query_ids)]
non_ae_counts = non_query_reac['pt'].value_counts().reset_index()
non_ae_counts.columns = ['Adverse_Event', 'Count']

# Get AE counts for cases where query drug is not the primary suspect 
non_ps_ids = non_ps_drug['primaryid'].unique()
non_ps_reac = reac[reac['primaryid'].isin(non_ps_ids)]
ps_non_ae_counts = non_ps_reac['pt'].value_counts().reset_index()
ps_non_ae_counts.columns = ['Adverse_Event', 'Count']

ae_counts.columns = ['Adverse_Event', 'Count_query_drug']
non_ae_counts.columns = ['Adverse_Event', 'Count_non_query_drug']
ps_ae_counts.columns = ['Adverse_Event', 'Count_query_drug']
ps_non_ae_counts.columns = ['Adverse_Event', 'Count_non_query_drug']
           
print(ae_counts.head(5))
print(non_ae_counts.head(5))



              Adverse_Event  Count_query_drug
0                    Nausea              5730
1                   Fatigue              5206
2              Constipation              4218
3  Platelet count decreased              3687
4                  Insomnia              2964
      Adverse_Event  Count_non_query_drug
0  Drug ineffective                595513
1     Off label use                392591
2           Fatigue                354688
3             Death                332021
4            Nausea                313484


In [25]:
ae_counts.head(5)

,Adverse_Event,Count_query_drug
0,Nausea,5730
1,Fatigue,5206
2,Constipation,4218
3,Platelet count decreased,3687
4,Insomnia,2964


In [26]:
# Merge AE counts for query drug and non-query drug
ae_comparison = pd.merge(
    ae_counts,
    non_ae_counts,
    on='Adverse_Event',
    how='outer'
)

# Filter to AEs with at least 3 reports for the query drug
ae_filtered = ae_comparison[
    (ae_comparison['Count_query_drug'].notna()) &
    (ae_comparison['Count_query_drug'] >= 3)
].copy()

print(ae_filtered.head(5)) 


                Adverse_Event  Count_query_drug  Count_non_query_drug
39        Abdominal adhesions              19.0                1517.0
41  Abdominal cavity drainage              11.0                 268.0
43       Abdominal discomfort             876.0               78489.0
44       Abdominal distension             493.0               41887.0
47           Abdominal hernia              66.0                2334.0


In [27]:
# Merge AE counts for primary-suspect query drug and non-query drug
ps_comparison = pd.merge(
    ps_ae_counts,
    ps_non_ae_counts,
    on='Adverse_Event',
    how='outer'
)

# Filter to AEs with at least 3 reports for the query drug
ps_filtered = ps_comparison[
    (ps_comparison['Count_query_drug'].notna()) &
    (ps_comparison['Count_query_drug'] >= 3)
].copy()

print(ps_filtered.head(5))



                Adverse_Event  Count_query_drug  Count_non_query_drug
39        Abdominal adhesions              19.0                1517.0
41  Abdominal cavity drainage              11.0                 268.0
43       Abdominal discomfort             866.0               78499.0
44       Abdominal distension             488.0               41892.0
47           Abdominal hernia              66.0                2334.0


In [28]:
# Calculates the number of reports that did not include a specific adverse event for both the query drug and non-query drugs
query_num = query_drug_ids.shape[0]
non_num = non_query_ids.shape[0]
ae_filtered['No_AE_query_drug'] = query_num - ae_filtered['Count_query_drug']
ae_filtered['No_AE_non_query_drug'] = non_num - ae_filtered['Count_non_query_drug']
ae_filtered


,Adverse_Event,Count_query_drug,Count_non_query_drug,No_AE_query_drug,No_AE_non_query_drug
39,Abdominal adhesions,19.0,1517.0,17168.0,8380575.0
41,Abdominal cavity drainage,11.0,268.0,17176.0,8381824.0
43,Abdominal discomfort,876.0,78489.0,16311.0,8303603.0
44,Abdominal distension,493.0,41887.0,16694.0,8340205.0
47,Abdominal hernia,66.0,2334.0,17121.0,8379758.0
...,...,...,...,...,...
19651,Wound haemorrhage,8.0,1798.0,17179.0,8380294.0
19653,Wound infection,6.0,4356.0,17181.0,8377736.0
19665,Wrist fracture,27.0,5122.0,17160.0,8376970.0
19686,Wrong technique in product usage process,11.0,108481.0,17176.0,8273611.0


In [29]:
# Calculates the number of reports that did not include a specific adverse event for both the PS query drug and non-query drugs 
ps_num = ps_ids.shape[0]
ps_non_num = non_ps_ids.shape[0]
ps_filtered['No_AE_query_drug'] = ps_num - ps_filtered['Count_query_drug']
ps_filtered['No_AE_non_query_drug'] = ps_non_num - ps_filtered['Count_non_query_drug']
ps_filtered


,Adverse_Event,Count_query_drug,Count_non_query_drug,No_AE_query_drug,No_AE_non_query_drug
39,Abdominal adhesions,19.0,1517.0,16767.0,8380976.0
41,Abdominal cavity drainage,11.0,268.0,16775.0,8382225.0
43,Abdominal discomfort,866.0,78499.0,15920.0,8303994.0
44,Abdominal distension,488.0,41892.0,16298.0,8340601.0
47,Abdominal hernia,66.0,2334.0,16720.0,8380159.0
...,...,...,...,...,...
19651,Wound haemorrhage,7.0,1799.0,16779.0,8380694.0
19653,Wound infection,6.0,4356.0,16780.0,8378137.0
19665,Wrist fracture,27.0,5122.0,16759.0,8377371.0
19686,Wrong technique in product usage process,10.0,108482.0,16776.0,8274011.0


In [35]:
# original contingency table construction with chi-squared p-value 
p_values = []

for _, row in ae_filtered.iterrows():
    table = [[row['Count_query_drug'], row['No_AE_query_drug']],
             [row['Count_non_query_drug'], row['No_AE_non_query_drug']]]
    _, p, _, _ = chi2_contingency(table)
    p_values.append(p)

ae_filtered['p_value'] = p_values


p_values = []

for _, row in ps_filtered.iterrows():
    table = [[row['Count_query_drug'], row['No_AE_query_drug']],
             [row['Count_non_query_drug'], row['No_AE_non_query_drug']]]

    _, p, _, _ = chi2_contingency(table)
    p_values.append(p)

ps_filtered['p_value'] = p_values
ps_filtered

,Adverse_Event,Count_query_drug,Count_non_query_drug,No_AE_query_drug,No_AE_non_query_drug,p_value
18,Abdominal discomfort,18.0,3877.0,877.0,395742.0,2.704608e-03
33,Abdominal pain,3.0,5228.0,892.0,394391.0,1.578875e-02
35,Abdominal pain upper,8.0,4665.0,887.0,394954.0,5.449742e-01
240,Acute myocardial infarction,3.0,354.0,892.0,399265.0,5.628653e-02
532,Anaemia,6.0,3594.0,889.0,396025.0,5.839123e-01
...,...,...,...,...,...,...
11742,Treatment failure,3.0,2049.0,892.0,397570.0,6.109124e-01
11878,Ulcer haemorrhage,4.0,126.0,891.0,399493.0,2.486020e-09
11936,Upper gastrointestinal haemorrhage,7.0,225.0,888.0,399394.0,8.858341e-17
12511,Vomiting,7.0,10233.0,888.0,389386.0,1.109142e-03


In [36]:
def phv_ebgm_qn(theta_hat, N, E):
    r1, b1, r2, b2, p = theta_hat
    prob_f1 = b1 / (b1 + E)
    prob_f2 = b2 / (b2 + E)
    f1_nb = nbinom.pmf(N, n=r1, p=prob_f1)
    f2_nb = nbinom.pmf(N, n=r2, p=prob_f2)
    num = p * f1_nb
    den = num + (1 - p) * f2_nb
    return num / den

def phv_ebgm_score(theta_hat, N, E, qn):
    r1, b1, r2, b2, _ = theta_hat
    e1 = digamma(r1 + N) - np.log(b1 + E)
    e2 = digamma(r2 + N) - np.log(b2 + E)
    exp_log = qn * e1 + (1 - qn) * e2
    eb_log2 = exp_log / np.log(2)
    return 2 ** eb_log2

def phv_ebgm_quant_bisect(cut_point, theta_hat, N, E, qn,
                          digits=2, limits=(-1e5, 1e5), max_iter=2000):
    r1, b1, r2, b2, _ = theta_hat
    lower, upper = limits
    tol = 0.5 * 10 ** (-digits)

    def post_cdf_minus_cp(x):
        c1 = gamma.cdf(x, a=r1 + N, scale=1/(b1 + E))
        c2 = gamma.cdf(x, a=r2 + N, scale=1/(b2 + E))
        return qn * c1 + (1 - qn) * c2 - cut_point

    N = np.asarray(N)
    E = np.asarray(E)
    qn = np.asarray(qn)

    guess = np.ones_like(N, dtype=float)
    err0 = post_cdf_minus_cp(guess)
    is_pos = err0 > 0
    left = np.where(is_pos, lower, guess)
    right = np.where(is_pos, guess, upper)

    for _ in range(max_iter):
        mid = (left + right) / 2
        err_mid = post_cdf_minus_cp(mid)
        if np.max((right - left) / 2) < tol:
            q = np.round(mid, digits)
            if np.any(q == upper):
                raise ValueError("increase maximum for 'limits'")
            return q
        err_left = post_cdf_minus_cp(left)
        same = np.sign(err_left) == np.sign(err_mid)
        left = np.where(same, mid, left)
        right = np.where(same, right, mid)

    raise RuntimeError("failed to converge -- try adjusting 'limits' or 'max_iter'")

def dbinbinom(x, size1, prob1, size2, prob2, w):
    return w * nbinom.pmf(x, n=size1, p=prob1) + (1 - w) * nbinom.pmf(x, n=size2, p=prob2)

def phvid_objective(theta, N, E):
    r1, b1, r2, b2, w = theta
    prob1 = b1 / (b1 + E)
    prob2 = b2 / (b2 + E)
    pmf = dbinbinom(N, r1, prob1, r2, prob2, w)
    return np.sum(-np.log(pmf + 1e-16))  # add tiny epsilon to avoid log(0)

def phv_ebgm(a, b, c, d, alpha=0.05, theta_init=None, squashing=True):
    """
    a, b, c, d: counts (scalars or array‐like)
    alpha: two‐sided significance level
    theta_init: array‐like [r1, b1, r2, b2, p] or pandas.DataFrame of guesses
    squashing: (not implemented – placeholder)
    """
    # turn inputs into arrays
    a, b, c, d = np.broadcast_arrays(a, b, c, d)
    if np.any(a < 0) or np.any(b < 0) or np.any(c < 0) or np.any(d < 0):
        raise ValueError("a, b, c, d must be non‐negative")
    # total counts
    tot = a + b + c + d
    N = a.astype(float)
    E = (a + b) / tot * (a + c)

    # squashing placeholder
    if squashing and np.any(a == 0):
        warnings.warn("squashing=True but no squash implemented; continuing with raw counts")

    # prepare initial guess
    if theta_init is None:
        x0 = np.array([0.2, 0.1, 2.0, 4.0, 1/3])
    else:
        if isinstance(theta_init, pd.DataFrame):
            x0 = theta_init.iloc[0].values
        else:
            x0 = np.asarray(theta_init, dtype=float)
        if x0.shape[0] != 5:
            raise ValueError("theta_init must have length 5")

    # bounds: r1,b1,r2,b2 > 0; p in (0,1)
    bounds = [(1e-6, None)]*4 + [(1e-6, 1-1e-6)]
    res = minimize(phvid_objective, x0, args=(N, E),
                   bounds=bounds, method='L-BFGS-B')
    if not res.success:
        raise RuntimeError("hyperparameter estimation failed: " + res.message)
    theta_hat = res.x

    # compute posterior weights, EBGM and CIs
    qn = phv_ebgm_qn(theta_hat, N, E)
    ebgm = phv_ebgm_score(theta_hat, N, E, qn)
    half = alpha / 2
    ci_low  = phv_ebgm_quant_bisect(half,      theta_hat, N, E, qn)
    ci_high = phv_ebgm_quant_bisect(1 - half,  theta_hat, N, E, qn)

    return pd.DataFrame({
        'ebgm':   ebgm,
        'ci_low': ci_low,
        'ci_high':ci_high
    })


In [44]:
from scipy.stats import chi2_contingency, gamma
import numpy as np

# Function to compute Odds Ratio (OR) and Confidence Interval (CI)
def compute_or_and_ci(a, b, c, d):
    # Add 0.5 to avoid zero counts
    a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5

    # Odds Ratio
    or_val = (a * d) / (b * c)

    # Log(OR) and standard error
    log_or = np.log(or_val)
    se = np.sqrt(1/a + 1/b + 1/c + 1/d)

    # Confidence Interval
    ci_low = np.exp(log_or - 1.96 * se)
    ci_high = np.exp(log_or + 1.96 * se)

    return or_val, ci_low, ci_high

# Function to compute Proportional Reporting Ratio (PRR), Standard Error (SE), and Confidence Interval (CI)
def compute_prr_and_ci(a, b, c, d):
    # Proportional Reporting Ratio (PRR)
    prr = (a / (a + b)) / (c / (c + d))

    # Standard Error (SE)
    se = np.sqrt(1/a + 1/c - 1/(a + b) - 1/(c + d))

    # Confidence Interval for PRR
    ln_prr = np.log(prr)
    ci_low = np.exp(ln_prr - 1.96 * se)
    ci_high = np.exp(ln_prr + 1.96 * se)

    return prr, se, ci_low, ci_high

# Function to add statistics (OR and PRR) to the DataFrame
def add_stats(df):
    # Add 0.5 to all columns to avoid zero counts
    a = df['Count_query_drug'] + 0.5
    b = df['No_AE_query_drug'] + 0.5
    c = df['Count_non_query_drug'] + 0.5
    d = df['No_AE_non_query_drug'] + 0.5

    # Compute Odds Ratio (OR) and Confidence Interval (CI) using vectorized operations
    or_vals = (a * d) / (b * c)
    log_or = np.log(or_vals)
    se_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    ci_low_or = np.exp(log_or - 1.96 * se_or)
    ci_high_or = np.exp(log_or + 1.96 * se_or)

    # Compute Proportional Reporting Ratio (PRR), SE, and Confidence Interval (CI) using vectorized operations
    prr_vals = (a / (a + b)) / (c / (c + d))
    se_prr = np.sqrt(1/a + 1/c - 1/(a + b) - 1/(c + d))
    ln_prr = np.log(prr_vals)
    ci_low_prr = np.exp(ln_prr - 1.96 * se_prr)
    ci_high_prr = np.exp(ln_prr + 1.96 * se_prr)

    # Perform chi-squared test for each row in a vectorized way
    p_values = df.apply(lambda row: chi2_contingency([[row['Count_query_drug'], row['No_AE_query_drug']], 
                                                      [row['Count_non_query_drug'], row['No_AE_non_query_drug']]])[1], axis=1)

    # Add new columns for OR, PRR, and p-value statistics
    df['odds_ratio'] = or_vals
    df['ci_lower_or'] = ci_low_or
    df['ci_upper_or'] = ci_high_or
    df['prr'] = prr_vals
    df['se_prr'] = se_prr
    df['ci_lower_prr'] = ci_low_prr
    df['ci_upper_prr'] = ci_high_prr
    df['p_value'] = p_values

    return df

ae_filtered = add_stats(ae_filtered)
ps_filtered = add_stats(ps_filtered) 

# Display the results
print(ae_filtered.head(10))


           Adverse_Event  Count_query_drug  Count_non_query_drug  \
13     Abdominal abscess               4.0                 105.0   
18  Abdominal discomfort             296.0                3599.0   
19  Abdominal distension             133.0                2158.0   
21      Abdominal hernia              16.0                  99.0   
26   Abdominal infection               3.0                  59.0   
29        Abdominal mass               6.0                  94.0   
33        Abdominal pain             309.0                4922.0   
34  Abdominal pain lower              23.0                 443.0   
35  Abdominal pain upper             273.0                4400.0   
37    Abdominal rigidity               3.0                  42.0   

    No_AE_query_drug  No_AE_non_query_drug       p_value  odds_ratio  \
13           12610.0              387795.0  9.706426e-01    1.311688   
18           12318.0              384301.0  3.699729e-57    2.569785   
19           12481.0              3

In [20]:
ps_filtered = ps_filtered.sort_values(by='odds_ratio', ascending=False)
ps_filtered

,Adverse_Event,Count_query_drug,Count_non_query_drug,No_AE_query_drug,No_AE_non_query_drug,p_value,odds_ratio,ci_lower_or,ci_upper_or,prr,se_prr,ci_lower_prr,ci_upper_prr,ebgm,ebgm05
3538,Drug effective for unapproved indication,504.0,1247.0,391.0,398372.0,0.000000e+00,411.507923,356.592356,474.880539,180.462586,0.040819,166.586930,195.493998,102.791983,95.385606
9572,Product use in unapproved indication,516.0,8430.0,379.0,391189.0,0.000000e+00,63.152770,55.218365,72.227280,27.330352,0.030607,25.739021,29.020067,24.629619,22.875390
9575,Product used for unknown indication,3.0,29.0,892.0,399590.0,9.718295e-20,53.119375,17.505669,161.185956,46.189829,0.605555,14.095742,151.357784,3.733056,1.275136
919,Arterial haemorrhage,3.0,31.0,892.0,399588.0,1.315782e-18,49.746467,16.455390,150.389080,43.209840,0.603715,13.233973,141.083128,3.717550,1.269839
9375,Pre-eclampsia,5.0,67.0,890.0,399552.0,2.462843e-27,36.559382,15.291927,87.404841,33.321021,0.462391,13.462417,82.473337,5.168434,2.250866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7801,Nausea,7.0,15544.0,888.0,384075.0,2.353524e-06,0.208566,0.101633,0.428007,0.201075,0.376566,0.096121,0.420627,0.223772,0.111349
3652,Dyspnoea,4.0,11856.0,891.0,387763.0,1.402044e-05,0.165083,0.065365,0.416926,0.150642,0.498963,0.056652,0.400563,0.181800,0.071635
952,Arthralgia,3.0,10982.0,892.0,388637.0,1.614822e-05,0.138772,0.048567,0.396519,0.121973,0.576459,0.039407,0.377533,0.156572,0.053482
4308,Fatigue,4.0,19074.0,891.0,380545.0,2.084178e-09,0.100704,0.039876,0.254316,0.093636,0.498931,0.035216,0.248966,0.114594,0.045154


In [76]:
from statsmodels.stats.multitest import multipletests
ae_filtered_valid = ae_filtered[ae_filtered['p_value'].notna()].copy()
_, corrected_p_values, _, _ = multipletests(ae_filtered_valid['p_value'], alpha=0.05, method='fdr_bh')

ae_filtered_valid['corrected_p_value'] = corrected_p_values

ps_filtered_valid = ps_filtered[ps_filtered['p_value'].notna()].copy()
_, corrected_p_values, _, _ = multipletests(ps_filtered_valid['p_value'], alpha=0.05, method='fdr_bh')

ps_filtered_valid['corrected_p_value'] = corrected_p_values

In [68]:
ps_filtered_valid = ps_filtered_valid.sort_values(by='odds_ratio', ascending=False)
ps_filtered_valid

,Adverse_Event,Count_query_drug,Count_non_query_drug,No_AE_query_drug,No_AE_non_query_drug,p_value,odds_ratio,ci_lower,ci_upper,corrected_p_value
3565,drug effective for unapproved indication,1191.0,2095.0,415.0,407148.0,0.000000e+00,557.170601,494.346265,627.979010,0.000000e+00
9723,product use in unapproved indication,1211.0,9453.0,395.0,399790.0,0.000000e+00,129.543842,115.433302,145.379250,0.000000e+00
8206,nsaid exacerbated respiratory disease,6.0,26.0,1600.0,409217.0,2.318625e-52,62.714217,26.556946,148.099597,2.782350e-51
9726,product used for unknown indication,9.0,49.0,1597.0,409194.0,6.858854e-68,49.159485,24.509982,98.598807,1.234594e-66
4468,foetal distress syndrome,3.0,24.0,1603.0,409219.0,1.523001e-13,36.457704,11.882653,111.857530,1.096561e-12
...,...,...,...,...,...,...,...,...,...,...
12655,vomiting,4.0,10129.0,1602.0,399114.0,1.515680e-08,0.110643,0.043853,0.279155,6.820561e-08
3679,dyspnoea,4.0,12378.0,1602.0,396865.0,1.361751e-10,0.090031,0.035685,0.227141,8.170504e-10
5194,headache,4.0,12788.0,1602.0,396455.0,5.740310e-11,0.087054,0.034505,0.219631,3.757294e-10
4334,fatigue,5.0,18946.0,1601.0,390297.0,2.976679e-16,0.070746,0.030624,0.163433,2.679011e-15
